# Case study 2: _Clostridium autoethanogenum_ metabolomics, proteomics and FBA

- The metabolomics comes from [Valgepea et al., 2018](https://doi.org/10.1186/s13068-018-1052-9).
- The proteomics was taken from [Valgepea et al., 2022](https://doi.org/10.1128/msystems.00026-22).

In [1]:
import json
from pathlib import Path

import cobra
import geckopy
import ggshu
import numpy as np
import pandas as pd
from cobra.sampling import sample

In [2]:
import os

In [3]:
model = cobra.io.read_sbml_model("../models/iCLAU786.xml")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [4]:
model.slim_optimize()

0.652234986862141

## 1. Flux sampling

We will use the Genome-scale Metabolic (GEM) model to perform flux sampling.

First, a function to merge two dataframes that we will use to join proteomics and fluxes and another function to prepare the experimental conditions of the GEM.

In [5]:
def apply_base_conditions(model: cobra.Model, exchange_groups_path: Path):
    """Apply base conditions in-place."""
    with open(exchange_groups_path) as file:
        reac_groups = json.load(file)
    for ex in reac_groups["influx"]:
        if ex in model.reactions:
            if model.reactions.get_by_id(ex).lower_bound < 0:
                model.reactions.get_by_id(ex).upper_bound = 0
            else:
                model.reactions.get_by_id(ex).bounds = 0, 0
    for ex in reac_groups["outflux"]:
        if ex in model.reactions:
            if model.reactions.get_by_id(ex).upper_bound > 0:
                model.reactions.get_by_id(ex).lower_bound = 0
            else:
                model.reactions.get_by_id(ex).bounds = 0, 0
    for ex in list(reac_groups["substrates"]) + list(reac_groups["aminoacids"]):
        if ex in model.reactions:
            model.reactions.get_by_id(ex).lower_bound = 0

Since there are three conditions, the model has to be constrained with the particular exchanges that represent the media and then sampled.

In preparation for the identifier translation, first we will get the identifiers on the map.

In [6]:
with open("../maps/cauto_map.json") as f:
    metabolic_map = json.load(f)["metabolism"]

In [7]:
reacs_in_map = {v["bigg_id"] for v in metabolic_map["reactions"].values()}

Let's gather the identifiers in the model.

In [8]:
model_to_map = {
    reac.id: reac.id
    if "bigg.reaction" not in reac.annotation
    else reac.annotation["bigg.reaction"]
    if isinstance(reac.annotation["bigg.reaction"], str)
    else reac.annotation["bigg.reaction"][0]
    for reac in model.reactions
}

In [9]:
map_to_model = {v: k for k, v in model_to_map.items()}

### CO

In [10]:
model.reactions.get_by_id("cpd11640_ext_b").bounds = (
    0.47,
    0.47,
)  #   H2
model.reactions.get_by_id("cpd00204_ext_b").bounds = -34.57, -34.57  # CO
model.reactions.get_by_id("cpd00084_ext_b").bounds = -0.07, 1000  # Cysteine
model.reactions.get_by_id("cpd00011_ext_b").bounds = 21.27, 21.27  # CO2
model.reactions.get_by_id("cpd00029_ext_b").bounds = 3.21, 1000  # Acetate
model.reactions.get_by_id("cpd00363_ext_b").bounds = 2.57, 1000  # Ethanol
model.reactions.get_by_id("cpd01947_ext_b").bounds = 0.06, 1000  # BDOH

In [11]:
samples = sample(model, n=400, processes=4)

In [12]:
samples.head()

,rxn12008_c0,rxn00541_c0,rxn00225_c0,rxn01199_c0,rxn10215_c0,rxn02483_c0,rxn00802_c0,rxn03638_c0,rxn06078_c0,rxn00868_c0,...,ProTrans_c0,IsoButTrans_c0,3MBTrans_c0,NaDiffusion_c0,PheTrans_c0,AsnTrans_c0,TyrTrans_c0,MetTrans_c0,rxn05469_c0,cpd00020_ext_b
0,1.089041e-17,0.268088,-735.916610,-0.000078,1.243577e-16,0.0,454.343558,0.000103,1.568404e-20,0.0,...,-61.553068,0.0,0.0,-526.588195,-0.000065,-0.000021,-0.000043,-129.650845,-0.000113,0.000113
1,4.771229e-17,0.417272,-715.242501,-0.002464,1.362107e-16,0.0,454.858929,0.000303,1.569091e-20,0.0,...,-82.326042,0.0,0.0,-505.135855,-0.000072,-0.000021,-0.000043,-149.874380,-0.003343,0.003343
2,2.560473e-16,0.413465,-720.626867,-0.007729,3.509706e-17,0.0,458.690769,0.000382,1.609110e-20,0.0,...,5.195525,0.0,0.0,-619.676740,-0.000687,-0.000022,-0.001479,-108.880477,-0.003457,0.003457
3,2.522121e-16,1.114675,-726.149210,-0.005396,6.261513e-17,0.0,454.871275,0.001172,1.547081e-20,0.0,...,-84.597524,0.0,0.0,-526.925758,-0.003407,-0.000269,-0.001456,-158.676984,-0.004654,0.004654
4,5.822410e-16,0.617041,-699.481599,-0.005455,1.841297e-14,0.0,476.362144,0.001246,1.570631e-20,0.0,...,-90.224897,0.0,0.0,-532.605359,-0.003027,-0.002765,-0.001469,-161.512559,-0.004693,0.004693


In [13]:
samples = samples.melt(var_name="reaction", value_name="flux")

samples["map_id"] = samples.reaction.apply(lambda x: model_to_map[x])

In [14]:
samples.head()

,reaction,flux,map_id
0,rxn12008_c0,1.089041e-17,rxn12008_c0
1,rxn12008_c0,4.771229e-17,rxn12008_c0
2,rxn12008_c0,2.560473e-16,rxn12008_c0
3,rxn12008_c0,2.522121e-16,rxn12008_c0
4,rxn12008_c0,5.822410e-16,rxn12008_c0


Filter by the reactions in the map (this is not actually needed, it just makes it easier to work with).

In [15]:
samples_map = samples[samples.map_id.isin(reacs_in_map)]

In [16]:
samples_map["condition"] = "CO"

/tmp/ipykernel_169084/1439376718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_map["condition"] = "CO"


### Syngas

In [17]:
model.reactions.get_by_id("cpd11640_ext_b").bounds = (
    -12.87,
    -12.87,
)  #   H2
model.reactions.get_by_id("cpd00204_ext_b").bounds = -31.56, -31.56  # CO
model.reactions.get_by_id("cpd00084_ext_b").bounds = -0.08, 1000  # Cysteine
model.reactions.get_by_id("cpd00011_ext_b").bounds = 12.91, 12.91  # CO2
model.reactions.get_by_id("cpd00029_ext_b").bounds = 4.25, 1000  # Acetate
model.reactions.get_by_id("cpd00363_ext_b").bounds = 3.74, 1000  # Ethanol
model.reactions.get_by_id("cpd01947_ext_b").bounds = 0.09, 1000  # BDOH

In [18]:
samples = sample(model, n=400, processes=4)
samples = samples.melt(var_name="reaction", value_name="flux")
samples["map_id"] = samples.reaction.apply(lambda x: model_to_map[x])
samples["condition"] = "Syngas"
samples_map = pd.concat([samples_map, samples[samples.map_id.isin(reacs_in_map)]])

### High-H2 CO

In [19]:
model.reactions.get_by_id("cpd11640_ext_b").bounds = (
    -32.52,
    -32.52,
)  #   H2
model.reactions.get_by_id("cpd00204_ext_b").bounds = -25.33, -25.33  # CO
model.reactions.get_by_id("cpd00084_ext_b").bounds = -0.09, 1000  # Cysteine
model.reactions.get_by_id("cpd00011_ext_b").bounds = 4.39, 4.39  # CO2
model.reactions.get_by_id("cpd00029_ext_b").bounds = 1.92, 1000  # Acetate
model.reactions.get_by_id("cpd00363_ext_b").bounds = 7.78, 1000  # Ethanol
model.reactions.get_by_id("cpd01947_ext_b").bounds = 0, 1000  # BDOH

In [20]:
samples = sample(model, n=400, processes=4)
samples = samples.melt(var_name="reaction", value_name="flux")
samples["map_id"] = samples.reaction.apply(lambda x: model_to_map[x])
samples["condition"] = "HighH2CO"
samples_map = pd.concat([samples_map, samples[samples.map_id.isin(reacs_in_map)]])

In [21]:
samples_map.head()

,reaction,flux,map_id,condition
800,rxn00225_c0,-735.916610,ACKr,CO
801,rxn00225_c0,-715.242501,ACKr,CO
802,rxn00225_c0,-720.626867,ACKr,CO
803,rxn00225_c0,-726.149210,ACKr,CO
804,rxn00225_c0,-699.481599,ACKr,CO


In [22]:
samples_map.shape

(72000, 4)

## 2. Metabolomics

This data comes from [Valgepea et al., 2018](https://biotechnologyforbiofuels.biomedcentral.com/articles/10.1186/s13068-018-1052-9) and [de Souza et al., 2018](https://link.springer.com/article/10.1007/s11306-018-1331-2).

The key transformation of the data is to translated the identifiers from the dataset
to the ones in the map.

In [23]:
df_met = pd.read_csv("../data/METABOLOMICS_Valgepea2018_format.tsv", sep="\t")

In [24]:
mets_in_map = {
    met["bigg_id"]
    for met in metabolic_map["nodes"].values()
    if met["node_type"] == "metabolite"
}

In [25]:
[f"{met}_c" for met in pd.unique(df_met.metabolite) if f"{met}_c" not in mets_in_map]

['3pg+2pg_c',
 '6pg_c',
 'acn_c',
 'cit+iso_c',
 'cmp_c',
 'ctp_c',
 'f16dp_c',
 'fad_c',
 'fmn_c',
 'fum_c',
 'ga3p_c',
 'gdp_c',
 'gmp_c',
 'gtp_c',
 'kga_c',
 'lac_c',
 'mal_c',
 'rl5p_c',
 'suc_c',
 'udp_c',
 'ump_c',
 'utp_c']

Of this, `3pg+2pg` is in the map but as separate metabolites and some others appear but with different identifiers.

In [26]:
df_met["map_id"] = df_met.metabolite.apply(lambda met: f"{met}_c")

In [27]:
df_met.map_id = df_met.map_id.replace(
    {
        "3pg+2pg_c": "3pg_c",
        "ga3p_c": "S_cpd00102_c",
        "lac_c": "lac__D_c",
        "rl5p_c": "ru5p_c",
        "6pg_c": "6pgc_c",
        "f16dp_c": "fdp_c",
    }
)

Add also the pool concentration for 2pg. This will be indicated in the figure.

In [28]:
df_met = pd.concat([df_met, df_met[df_met.map_id == "3pg_c"].assign(map_id="2pg_c")])

## 3. Proteomics

To map the model to the proteins, I will use the enzyme constraint model since it provides a structural way to associate proteins with reactions.

In [29]:
import geckopy

In [31]:
ec_model = geckopy.io.read_sbml_ec_model("../models/eciCLAU786.xml")

/home/georg/.virtualenvs/gecko/lib/python3.9/site-packages/cobra/core/group.py:148: UserWarning: need to pass in a list
  warn("need to pass in a list")
Metabolite prot_U5RTZ1 does not use Uniprot ID.
Metabolite prot_U5RYC1 does not use Uniprot ID.
Metabolite prot_U5RRE0 does not use Uniprot ID.
Metabolite prot_U5RS90 does not use Uniprot ID.
Metabolite prot_U5RXZ7 does not use Uniprot ID.
Metabolite prot_U5S114 does not use Uniprot ID.
Metabolite prot_A0A3M0SRQ1 does not use Uniprot ID.
Metabolite prot_U5RYX9 does not use Uniprot ID.
Metabolite prot_U5S2A9 does not use Uniprot ID.
Metabolite prot_U5S1H6 does not use Uniprot ID.
Metabolite prot_U5RTD1 does not use Uniprot ID.
Metabolite prot_CAETHG_RS09960 does not use Uniprot ID.
Metabolite prot_CAETHG_RS09965 does not use Uniprot ID.
Metabolite prot_CAETHG_RS13725 does not use Uniprot ID.
Metabolite prot_CAETHG_RS13765 does not use Uniprot ID.
Metabolite prot_CAETHG_RS13770 does not use Uniprot ID.
Metabolite prot_CAETHG_RS13745 does

Formatted version of the excel file provided at the Supplementary material of [Valgepea et al., 2022](https://journals.asm.org/doi/full/10.1128/msystems.00026-22?rfr_dat=cr_pub++0pubmed&url_ver=Z39.88-2003&rfr_id=ori%3Arid%3Acrossref.org).

In [32]:
df_prot = pd.read_csv("../data/absprot_converted.tsv", sep="\t")

The data is index by the condition and gene ID. The latter is the name of the proteins in the model.

In [33]:
df_prot.head()

,Gene/proteinID,Brown et al. 2014 ID,Description of gene/protein product,Proposed gene name,condition,nmol/gDCW,SD,mM,logSD
0,CAETHG_RS00020,CAETHG_0004,Glyoxylate reductase,hprA,CO,1.9,0.5,0.000782,0.258765
1,CAETHG_RS00020,CAETHG_0004,Glyoxylate reductase,hprA,High-H2 CO,1.7,0.9,0.000700,0.497068
2,CAETHG_RS00020,CAETHG_0004,Glyoxylate reductase,hprA,Syngas,1.4,0.5,0.000576,0.346479
3,CAETHG_RS00050,CAETHG_0011,diguanylate phosphodiesterase metal dependent h,RS00050,CO,NaN,NaN,NaN,NaN
4,CAETHG_RS00050,CAETHG_0011,diguanylate phosphodiesterase metal dependent h,RS00050,High-H2 CO,0.3,0.1,0.000123,0.324593


Since most of the proteins mapped to a reaction appear to be a complex (in opposition of isozymes), let's just assign them to the lowest value of the participating enzymes.

In [34]:
model_to_map_f = {k: v for k, v in model_to_map.items() if v in reacs_in_map}

In [35]:
gene_to_reac = {
    prot.annotation["gene"]: [
        model_to_map[reac.id] for reac in prot.reactions if reac.id in model_to_map_f
    ]
    for prot in ec_model.proteins
}

In [36]:
gene_to_reac = {k: v for k, v in gene_to_reac.items() if v}

In [37]:
df_prot_map = df_prot.loc[df_prot["Gene/proteinID"].isin(gene_to_reac.keys()), :]

In [38]:
reac_to_gene = [(reac, k) for k, v in gene_to_reac.items() for reac in v]

In [39]:
reac_to_genes = {}
for reac, gene in reac_to_gene:
    if reac in reac_to_genes:
        reac_to_genes[reac].append(gene)
    else:
        reac_to_genes[reac] = [gene]

In [40]:
df_prot_map["reaction"] = None

/tmp/ipykernel_169084/1985740672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prot_map["reaction"] = None


In [41]:
df_prot_map.loc[df_prot_map.mM.isna(), "mM"] = 0.0

In [42]:
df_prot_map.loc[
    df_prot_map["Gene/proteinID"].isin(["CAETHG_RS13725", "CAETHG_RS13765"]), :
].min()["Gene/proteinID"]

'CAETHG_RS13725'

In [43]:
for reac, genes in reac_to_genes.items():
    protein = df_prot_map.loc[df_prot_map["Gene/proteinID"].isin(genes), :].min()[
        "Gene/proteinID"
    ]
    df_prot_map.loc[df_prot_map["Gene/proteinID"] == protein, "reaction"] = reac

In [44]:
df_prot_map_sel = df_prot_map[["reaction", "condition", "mM"]]

In [45]:
df_prot_map_sel.condition = df_prot_map_sel.condition.str.replace(" ", "").str.replace(
    "-", ""
)

/tmp/ipykernel_169084/1328051487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prot_map_sel.condition = df_prot_map_sel.condition.str.replace(" ", "").str.replace(


In [46]:
df_met.head()

,metabolite,condition,avg,log scale sigma,mM,map_id
0,3pg+2pg,CO,0.000220,0.0323214708714787,0.000220,3pg_c
1,3pg+2pg,HighH2CO,0.000254,0.0215937379844198,0.000254,3pg_c
2,3pg+2pg,Syngas,0.000473,0.0976187084056288,0.000473,3pg_c
3,5mthf,CO,0.000037,1.0,0.000037,5mthf_c
4,5mthf,HighH2CO,0.000037,1.0,0.000037,5mthf_c


### ggshu

Once all the data is tidied, ggshu can be used to generate the input for shu.

We want three mappings:

1. Enzyme concentrations for each condition as coloured boxes at the right hand side of the reaction.
2. Flux density funcions for each condition at the left hand side of the reaction.
3. Use the arrow color to indicate if there is data (1 and 2) for that particular reaction.
4. Concentration as the color and size of the reactions themselves.

In [47]:
samples_map.reaction = samples_map.map_id

Utility function to perform the merge of enzyme concentrations and fluxes.

In [48]:
def select_group_no_na(y: pd.Series, var: str):
    x = y.copy()
    if not x[var].isna().all():
        x[var] = x[var].dropna().iloc[0]
    return x

Collapse rows with only mM into the flux rows.

In [49]:
df_reac = (
    pd.concat(
        [df_prot_map_sel, samples_map],
        keys=["condition", "reaction"],
        join="outer",
    )
    .groupby(["reaction", "condition"], group_keys=False)
    .apply(select_group_no_na, var="mM")
    .reset_index(drop=True)
)

In [50]:
df_reac["has_data"] = ~(df_reac.mM.isna() & df_reac.flux.isna())
df_reac.has_data = df_reac.has_data.astype(int)

In [51]:
df_reac

,reaction,condition,mM,flux,map_id,has_data
0,leq000004_c0,CO,0.091704,NaN,NaN,1
1,leq000004_c0,HighH2CO,0.094833,NaN,NaN,1
2,leq000004_c0,Syngas,0.071701,NaN,NaN,1
3,PGI,CO,0.001194,NaN,NaN,1
4,PGI,HighH2CO,0.000000,NaN,NaN,1
...,...,...,...,...,...,...
72034,cpd00221_ext_b,HighH2CO,NaN,0.001045,cpd00221_ext_b,1
72035,cpd00221_ext_b,Syngas,NaN,0.000664,cpd00221_ext_b,1
72036,cpd00221_ext_b,CO,NaN,0.009752,cpd00221_ext_b,1
72037,cpd00221_ext_b,HighH2CO,NaN,0.010361,cpd00221_ext_b,1


In [53]:
(
    (
        ggshu.ggmap(
            df_reac,
            ggshu.aes(reaction="reaction", color="mM", condition="condition", y="flux"),
        )
        + ggshu.geom_boxpoint()
        + ggshu.geom_kde(side="left")
        #+ ggshu.geom_arrow(aes=ggshu.aes(color="has_data", size="has_data"))
    )
    # instead of using pandas to merge the above dataframe with the metabolite concentrations
    # we use the ggshu operator "/" to merge the two resulting grammar objects for convenience
    / (
        ggshu.ggmap(
            df_met,
            ggshu.aes(
                metabolite="map_id", color="mM", size="mM", condition="condition"
            ),
        )
        + ggshu.geom_metabolite()
    )
).to_json("../data/omics")

Geom data coerced distribution data to means.
Geom data coerced distribution data to means.
Geom data coerced distribution data to means.
